In [1]:
import torch
import pyqcu
from pyqcu.dslash import give_clover, make_clover, give_wilson
from pyqcu.lattice import generate_gauge_field
from pyqcu.solver import multigrid
from pyqcu import _torch
kappa = 0.125
path = pyqcu.__file__.replace('pyqcu/__init__.py', 'examples/data/')
refer_U = torch.zeros(3, 3, 4, 8, 8, 8, 8, dtype=torch.complex64, device=torch.device(
    'cuda' if torch.cuda.is_available() else 'cpu'))
generate_gauge_field(refer_U, seed=42, sigma=0.1, verbose=False)
refer_x = _torch.randn(size=(
    4, 3, refer_U.shape[-4], refer_U.shape[-3], refer_U.shape[-2], refer_U.shape[-1]), dtype=refer_U.dtype, device=refer_U.device)
clover_term = make_clover(U=refer_U, kappa=kappa, verbose=True)
refer_b = give_clover(src=refer_x, clover_term=clover_term, verbose=True) + \
    give_wilson(src=refer_x, U=refer_U, kappa=kappa,
                with_I=True, verbose=True)
mg = multigrid(dtype_list=[refer_U.dtype]*10, device_list=[refer_U.device]*10, U=refer_U,
               clover_term=clover_term, kappa=kappa, tol=1e-6, max_iter=1000, max_levels=2, verbose=False)
mg.init()


PYQCU::DSLASH::CLOVER:
 Applying Dirac operator...
PYQCU::DSLASH::CLOVER:
 Gauge field shape: torch.Size([3, 3, 4, 8, 8, 8, 8])
PYQCU::DSLASH::CLOVER:
 Processing xy-direction (mu=-4,nu=-3)...
PYQCU::DSLASH::CLOVER:
 sigmaF term norm: 274.1340026855469
PYQCU::DSLASH::CLOVER:
 Processing xz-direction (mu=-4,nu=-2)...
PYQCU::DSLASH::CLOVER:
 sigmaF term norm: 272.94403076171875
PYQCU::DSLASH::CLOVER:
 Processing xt-direction (mu=-4,nu=-1)...
PYQCU::DSLASH::CLOVER:
 sigmaF term norm: 274.02032470703125
PYQCU::DSLASH::CLOVER:
 Processing yz-direction (mu=-3,nu=-2)...
PYQCU::DSLASH::CLOVER:
 sigmaF term norm: 276.39276123046875
PYQCU::DSLASH::CLOVER:
 Processing yt-direction (mu=-3,nu=-1)...
PYQCU::DSLASH::CLOVER:
 sigmaF term norm: 274.94671630859375
PYQCU::DSLASH::CLOVER:
 Processing zt-direction (mu=-2,nu=-1)...
PYQCU::DSLASH::CLOVER:
 sigmaF term norm: 275.265380859375
PYQCU::DSLASH::CLOVER:
 Clover term complete
PYQCU::DSLASH::CLOVER:
 clover norm: 10.51058292388916
PYQCU::DSLASH::CLOV

In [ ]:
mg.op_list[1].matvec

<bound method op.matvec of <pyqcu.solver._multigrid.op object at 0x7f2a05fe7490>>

In [4]:
index = 0
null_vecs = mg.nv_list[index]
matvec = mg.op_list[index].matvec
for i in range(null_vecs.shape[0]):
    print(
        f"(matvec(null_vecs[i])/null_vecs[i]).flatten()[:10]:{(matvec(null_vecs[i])/null_vecs[i]).flatten()[:10]}")

(matvec(null_vecs[i])/null_vecs[i]).flatten()[:10]:tensor([-0.0035-0.0082j, -0.0522+0.0026j,  0.0193-0.0288j,  0.0125+0.0031j,
         0.0203-0.0814j,  0.0660+0.0171j, -0.0394+0.0077j,  0.0672-0.0085j,
        -0.0295+0.0210j,  0.0212-0.0127j], device='cuda:0')
(matvec(null_vecs[i])/null_vecs[i]).flatten()[:10]:tensor([ 0.0468+0.1275j,  0.0768+0.0067j,  0.0734+0.0324j,  0.1257-0.0118j,
        -0.0294+0.0496j,  0.0207+0.0684j,  0.0414+0.0165j,  0.0410+0.0381j,
         0.0650+0.0555j,  0.0463+0.0914j], device='cuda:0')
(matvec(null_vecs[i])/null_vecs[i]).flatten()[:10]:tensor([ 0.0476-0.0043j,  0.0485+0.0149j,  0.0386+0.0052j,  0.0197-0.0315j,
         0.0307-0.0071j, -0.0373+0.0187j,  0.0237+0.0861j,  0.0085+0.0022j,
         0.0711+0.0036j, -0.0405-0.0008j], device='cuda:0')
(matvec(null_vecs[i])/null_vecs[i]).flatten()[:10]:tensor([ 0.0584+0.0200j,  0.1044-0.1217j,  0.0038+0.2566j,  0.0023+0.0046j,
         0.0427+0.0137j, -0.0546+0.1785j,  0.0624+0.1690j,  0.0291+0.0071j,
        

In [ ]:

x = mg.solve(b=refer_b)
mg.plot()